<a href="https://colab.research.google.com/github/raheelam98/LangChain_Fundamentals/blob/main/langchain_course_jupyter/section_3_data_processing/part4_custom_search_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Custom Search Agent**

In [54]:
# Install the required packages:
%%capture --no-stderr
%pip install -U  langsmith # check
%pip install --quiet -U  langchain_google_genai langchain_core langchain langgraph   python-dotenv langsmith

In [55]:
%%capture --no-stderr
%pip install ChatGoogleGenerativeAI

In [48]:
# packages
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [56]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ice_breaker"

In [57]:
# API Keys
# Get the GEMINI API key from user data
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [40]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=gemini_api_key, temperature=0)

In [58]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0
)
print("Google Generative AI model initialized!")

Google Generative AI model initialized!


[Tavily Search](https://python.langchain.com/docs/integrations/tools/tavily_search/)

[TAVILY_API_KEY](https://tavily.com/)


In [59]:
# API keys set up
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

[hwchase17/react](https://smith.langchain.com/hub/hwchase17/react)



**Create React Agent**

In [60]:
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import (create_react_agent, AgentExecutor)
from langchain import hub

from langchain_google_genai import ChatGoogleGenerativeAI

# from dotenv import load_dotenv

# load_dotenv()

In [61]:
from langchain_community.tools.tavily_search import TavilySearchResults


def get_profile_url_tavily(name: str):
    """Searches for Linkedin or Twitter Profile Page."""
    search = TavilySearchResults()
    res = search.run(f"{name}")
    return res

In [62]:
from dotenv import load_dotenv

load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.prompt import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import (
    create_react_agent,
    AgentExecutor,
)
from langchain import hub
# from tools.tools import get_profile_url_tavily


def lookup(name: str) -> str:
    llm = ChatGoogleGenerativeAI(
        temperature=0,
        model="gemini-1.5-flash",
    )
    template = """given the full name {name_of_person} I want you to get it me a link to their Linkedin profile page.
                              Your answer should contain only a URL"""

    prompt_template = PromptTemplate(
        template=template, input_variables=["name_of_person"]
    )
    tools_for_agent = [
        Tool(
            name="Crawl Google 4 linkedin profile page",
            func=get_profile_url_tavily,
            description="useful for when you need get the Linkedin Page URL",
        )
    ]

    react_prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(llm=llm, tools=tools_for_agent, prompt=react_prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools_for_agent, verbose=True)

    result = agent_executor.invoke(
        input={"input": prompt_template.format_prompt(name_of_person=name)}
    )

    linked_profile_url = result["output"]
    return linked_profile_url


# if __name__ == "__main__":
#     print(lookup(name="Eden Marco Udemy"))

In [64]:
# if __name__ == "__main__":
#     print(lookup(name="Eden Marco Udemy"))